# Retail Customer Churn – Data Preparation (Python)

### Objective
Prepare a clean and standardized customer level dataset to enable SQL based churn analysis of churn patterns, revenue impact and customer risk segmentation.

### Key Questions
- How many customers are churned, at risk, or active?
- Where is churn related revenue concentrated?
- Which customer segments should be prioritized for retention?

## Data Loading & Structural Inspection

In [6]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [7]:
df = pd.read_excel('../data/retail_churn.xlsx')


### Initial Data Inspection

Reviewing dataset structure, data types and basic statistics to identify potential quality issues before cleaning.

In [8]:
df.shape

(525461, 8)

In [9]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [11]:
df.describe(include='all')

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
count,525461.0,525461,522533,525461.000000,525461,525461.000000,417534.000000,525461
unique,28816.0,4632,4681,NaN,NaN,NaN,NaN,40
top,537434.0,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,NaN,NaN,NaN,United Kingdom
freq,675.0,3516,3549,NaN,NaN,NaN,NaN,485852
mean,NaN,NaN,NaN,10.337667,2010-06-28 11:37:36.845017856,4.688834,15360.645478,NaN
min,NaN,NaN,NaN,-9600.000000,2009-12-01 07:45:00,-53594.360000,12346.000000,NaN
25%,NaN,NaN,NaN,1.000000,2010-03-21 12:20:00,1.250000,13983.000000,NaN
50%,NaN,NaN,NaN,3.000000,2010-07-06 09:51:00,2.100000,15311.000000,NaN
75%,NaN,NaN,NaN,10.000000,2010-10-15 12:45:00,4.210000,16799.000000,NaN
max,NaN,NaN,NaN,19152.000000,2010-12-09 20:01:00,25111.090000,18287.000000,NaN


## Data Cleaning & Standardization

In [12]:
def clean_string(value):
    if isinstance(value, str):
        value = value.strip()
        value = value.title()
    return value

In [13]:
df['Country']=df['Country'].apply(clean_string)

Standardizing categorical text fields to ensure consistency for grouping and filtering in SQL analysis.

In [14]:
df.columns = (df.columns
      .str.strip()
      .str.lower()
      .str.replace(' ', '_'))

In [15]:
df = df.rename(columns={'stockcode': 'stock_code','invoicedate': 'invoice_date'})

In [16]:
df.dtypes

invoice                 object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
price                  float64
customer_id            float64
country                 object
dtype: object

In [17]:
df['invoice_date']= pd.to_datetime(df['invoice_date'], errors = 'coerce')

In [18]:
df = df[df['customer_id'].notna()]

In [19]:
df['customer_id']=df['customer_id'].astype(int)

In [20]:
df = df[(df['quantity']>0) & (df['price']>0)] 

In [21]:
df['revenue']=df['quantity']*df['price']

In [22]:
df.dtypes

invoice                 object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
price                  float64
customer_id              int64
country                 object
revenue                float64
dtype: object

## Exploratory Data Analysis (EDA)

In [23]:
df['invoice_date'].min(), df['invoice_date'].max()

(Timestamp('2009-12-01 07:45:00'), Timestamp('2010-12-09 20:01:00'))

In [24]:
df['customer_id'].nunique()

4312

In [25]:
df['revenue'].sum()

np.float64(8832003.274000002)

In [26]:
df['revenue'].describe()

count    407664.000000
mean         21.664909
std          77.150058
min           0.001000
25%           4.950000
50%          11.900000
75%          19.500000
max       15818.400000
Name: revenue, dtype: float64

In [27]:
customer_df = (df.groupby('customer_id').
               agg(first_purchase_date=('invoice_date', 'min'),
                   last_purchase_date=('invoice_date', 'max')).
reset_index())

In [28]:
customer_df.sample(5)

,customer_id,first_purchase_date,last_purchase_date
4263,18225,2009-12-04 16:52:00,2010-12-09 15:46:00
209,12681,2009-12-13 13:31:00,2010-12-07 10:08:00
3103,16648,2010-04-28 14:58:00,2010-09-10 11:59:00
3774,17553,2010-04-08 09:44:00,2010-08-24 09:49:00
2932,16408,2010-04-27 14:08:00,2010-04-27 14:08:00


In [29]:
analysis_end_date = df['invoice_date'].max()
analysis_end_date

Timestamp('2010-12-09 20:01:00')

In [30]:
customer_df['days_since_last_purchase']=(analysis_end_date - customer_df['last_purchase_date']).dt.days

Descriptive Analysis of Customer Inactivity

In [31]:
customer_df['days_since_last_purchase'].describe(include='all')

count    4312.000000
mean       90.171846
std        96.860633
min         0.000000
25%        17.000000
50%        52.000000
75%       135.000000
max       373.000000
Name: days_since_last_purchase, dtype: float64

In [32]:
customer_df['days_since_last_purchase'].quantile([0.5,0.6,0.7,0.8,0.9])

0.5     52.0
0.6     70.0
0.7    105.0
0.8    175.0
0.9    253.9
Name: days_since_last_purchase, dtype: float64

Operational Definition of Customer Churn

Business Question: At what point of customer inactivity can a customer be reliably classified as churned, given the observed purchasing behavior?

In [33]:
churn_threshold_days = 175
customer_df['churn_eligible'] = ((analysis_end_date - customer_df['first_purchase_date']).dt.days> churn_threshold_days)

In [34]:
customer_df['churn_eligible'].value_counts()

churn_eligible
True     2866
False    1446
Name: count, dtype: int64

In [35]:
customer_df['churn_eligible'].value_counts(normalize=True)

churn_eligible
True     0.664657
False    0.335343
Name: proportion, dtype: float64

In [36]:
def classify_customers(days):
    if days <70:
        return 'Active'
    elif days< churn_threshold_days:
        return 'At risk'
    else:
        return 'Churned'

In [37]:
customer_df['customer_status']=customer_df['days_since_last_purchase'].apply(classify_customers)

In [38]:
customer_df['customer_status'].value_counts()

customer_status
Active     2584
At risk     864
Churned     864
Name: count, dtype: int64

In [39]:
customer_df['customer_status'].value_counts(normalize = True)

customer_status
Active     0.599258
At risk    0.200371
Churned    0.200371
Name: proportion, dtype: float64

Customer Value and Purchase Behavior


In [40]:
customer_revenue = (df.groupby ('customer_id')['revenue'].sum().reset_index()).rename(columns={'revenue':'total_revenue'})

In [41]:
customer_frequency = (df.groupby('customer_id')['invoice'].nunique().reset_index()).rename(columns={'invoice':'total_invoice'})

In [42]:
customer_df = customer_df.merge(customer_revenue,on='customer_id',how='left')
customer_df = customer_df.merge(customer_frequency,on='customer_id',how='left')

In [43]:
customer_df.columns

Index(['customer_id', 'first_purchase_date', 'last_purchase_date',
       'days_since_last_purchase', 'churn_eligible', 'customer_status',
       'total_revenue', 'total_invoice'],
      dtype='object')

In [44]:
customer_df.isna().sum()

customer_id                 0
first_purchase_date         0
last_purchase_date          0
days_since_last_purchase    0
churn_eligible              0
customer_status             0
total_revenue               0
total_invoice               0
dtype: int64

In [45]:
customer_df['total_revenue'].describe()

count      4312.000000
mean       2048.238236
std        8914.481280
min           2.950000
25%         307.987500
50%         706.020000
75%        1723.142500
max      349164.350000
Name: total_revenue, dtype: float64

In [46]:
customer_df['total_invoice'].describe()

count    4312.000000
mean        4.455705
std         8.170213
min         1.000000
25%         1.000000
50%         2.000000
75%         5.000000
max       205.000000
Name: total_invoice, dtype: float64

In [47]:
customer_df[['customer_id','total_revenue','total_invoice','customer_status']].head()

,customer_id,total_revenue,total_invoice,customer_status
0,12346,372.86,11,At risk
1,12347,1323.32,2,Active
2,12348,222.16,1,At risk
3,12349,2671.14,3,Active
4,12351,300.93,1,Active


In [48]:
churn_summary = (customer_df.groupby('customer_status')[['total_revenue','total_invoice']].mean())
churn_summary

,total_revenue,total_invoice
customer_status,,
Active,2842.472190,5.958978
At risk,1108.312663,2.820602
Churned,612.815965,1.594907


## Data Modeling for SQL

In [49]:
transactions_df = df[[
'invoice',
'stock_code',
'description',
'quantity',
'price',
'invoice_date',
'customer_id',
'country']]

## Data Quality Validation

In [50]:
customer_df.shape
transactions_df.shape

(407664, 8)

In [51]:
customer_df['customer_id'].is_unique

True

In [52]:
customer_df['customer_id'].isna().sum()
transactions_df['customer_id'].isna().sum()

np.int64(0)

In [53]:
customer_df.dtypes
transactions_df.dtypes

invoice                 object
stock_code              object
description             object
quantity                 int64
price                  float64
invoice_date    datetime64[ns]
customer_id              int64
country                 object
dtype: object

In [54]:
customer_df.head()
transactions_df.head()

,invoice,stock_code,description,quantity,price,invoice_date,customer_id,country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95,2009-12-01 07:45:00,13085,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,6.75,2009-12-01 07:45:00,13085,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,6.75,2009-12-01 07:45:00,13085,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10,2009-12-01 07:45:00,13085,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25,2009-12-01 07:45:00,13085,United Kingdom


## Load Data into MySQL


In [ ]:
engine = create_engine('mysql+mysqlconnector://<user>:<password>@localhost:3306/<database>')


In [57]:
customer_df.to_sql(
    name='customers',
    con=engine,
    if_exists='append',
    index=False)

4312

In [58]:
transactions_df.to_sql(
    name='transactions',
    con=engine,
    if_exists='append',
    index=False)

407664

## Next Step

The datasets created in this notebook serve as the input for the SQL churn analysis, where customer behavior, churn logic and revenue impact are evaluated as we mentioned at the beggining of the notebook.
